In [ ]:
# 상위 문서 검색기(ParentDocumentRetriever)
#문서 검색과 문서 분할의 균형 잡기
# 문서 검색 과정에서 문서를 적절한 크기의 조각(청크)으로 나누는 것은 다음의 상충되는 두 가지 중요한 요소를 고려해야 합니다.
# -작은 문서를 원하는 경우: 이렇게 하면 문서의 임베딩이 그 의미를 가장 정확하게 반영할 수 있습니다. 문서가 너무 길면 임베딩이 의미를 잃어버릴 수 있습니다.
# - 각 청크의 맥락이 유지되도록 충분히 긴 문서를 원하는 경우입니다.
#
# ParentDocumentRetriever의 역할
# 이 두 요구 사항 사이의 균형을 맞추기 위해 ParentDocumentRetriever라는 도구가 사용됩니다. 
# 이 도구는 문서를 작은 조각으로 나누고, 이 조각들을 관리합니다. 검색을 진행할 때는, 먼저 이 작은 조각들을 찾아낸 다음, 
# 이 조각들이 속한 원본 문서(또는 더 큰 조각)의 식별자(ID)를 통해 전체적인 맥락을 파악할 수 있습니다.
# 여기서 '부모 문서'란, 작은 조각이 나누어진 원본 문서를 말합니다. 
# 이는 전체 문서일 수도 있고, 비교적 큰 다른 조각일 수도 있습니다. 
# 이 방식을 통해 문서의 의미를 정확하게 파악하면서도, 전체적인 맥락을 유지할 수 있게 됩니다.
#
# 정리
# - 문서 간의 계층 구조 활용: ParentDocumentRetriever는 문서 검색의 효율성을 높이기 위해 문서 간의 계층 구조를 활용합니다.
# - 검색 성능 향상: 관련성 높은 문서를 빠르게 찾아내며, 주어진 질문에 대한 가장 적합한 답변을 제공하는 문서를 효과적으로 찾아낼 수 있습니다. 
# 문서를 검색할 때 자주 발생하는 두 가지 상충되는 요구 사항이 있습니다

%pip install -qU deeplake

In [ ]:
# 루트경로에 .env 파일을 만들고, OPENAI_API_KEY='{API_KEY}' 식으로 입력한다.
# API 키를 환경변수로 관리하기 위한 .env설정 파일 로딩
import os
from dotenv import load_dotenv

load_dotenv() # API 키 정보 로드
print(f"[OPENAI_API_KEY]\n{os.environ['OPENAI_API_KEY']}\n")
print(f"[HUGGINGFACEHUB_API_TOKEN]\n{os.environ['HUGGINGFACEHUB_API_TOKEN']}")

In [ ]:
from langchain.storage import InMemoryStore
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever

loaders = [
    # 파일을 로드합니다.
    TextLoader("../data/company/03.경조사지원규정_12.01.01.txt"),
    TextLoader("../data/company/04.급여규정_16.06.01.txt"),
    TextLoader("../data/company/08.보안관리규정_11.10.01.txt"),
]
docs = []  # 빈 리스트를 생성합니다.
for loader in loaders:  # loaders 리스트의 각 로더에 대해 반복합니다.
    docs.extend(loader.load())  # 로더를 사용하여 문서를 로드하고 docs 리스트에 추가합니다.

# 자식 분할기 생성
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

# DB 생성
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=OpenAIEmbeddings()
)

store = InMemoryStore()

# Retriever 생성
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

# retriever.add_documents(docs, ids=None) 함수로 문서목록을 추가합니다.
# - ids 가 None 이면 자동으로 생성됩니다.
# - add_to_docstore=False 로 설정시 document 를 중복으로 추가하지 않습니다. 단, 중복을 체크하기 위한 ids 값이 필수 값으로 요구됩니다.
retriever.add_documents(docs, ids=None, add_to_docstore=True)


In [ ]:
# 유사도 검색 수행
query = "아버지 돌아가시면 유급휴가는 며칠인가요?"
sub_docs = vectorstore.similarity_search(query)
for sub_doc in sub_docs:
    print(f'{sub_doc.page_content}', end="\n=================================\n")

In [ ]:
# 문서를 검색하여 가져옵니다.
retrieved_docs = retriever.get_relevant_documents(query)

print(f'*검색된 문서 계수:{len(retrieved_docs)}',end="\n\n=====================\n\n")

# 검색된 문서의 문서의 페이지 내용의 길이를 출력합니다.
for i, retrieved_doc in enumerate(retrieved_docs): 
    retrieved_doc_len = len(retrieved_doc.page_content)
    print(f"*문서{i+1}의 길이: {retrieved_doc_len}", end="\n\n\n")
    # 문서의 일부를 출력합니다.
    print(retrieved_doc.page_content[:100], end="\n\n=====================\n\n")


In [2]:
# 방법 2: 문서도 청크로 나누자
# 위 문제점은 문서가 너무 길면 안됨.
# => 문서도 청크로 나누자.
#
from langchain.storage import InMemoryStore
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever

loaders = [
    # 파일을 로드합니다.
    TextLoader("../data/company/03.경조사지원규정_12.01.01.txt"),
    TextLoader("../data/company/04.급여규정_16.06.01.txt"),
    TextLoader("../data/company/08.보안관리규정_11.10.01.txt"),
]
docs = []  # 빈 리스트를 생성합니다.
for loader in loaders:  # loaders 리스트의 각 로더에 대해 반복합니다.
    docs.extend(loader.load())  # 로더를 사용하여 문서를 로드하고 docs 리스트에 추가합니다.

# 자식 분할기 생성
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)

# 부모 문서를 생성하는 데 사용되는 텍스트 분할기입니다.
# => child_splitter 보다는 사이즈 큰 chunk로 나눔.
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

# DB 생성
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=OpenAIEmbeddings()
)

store = InMemoryStore()

# retriever 재 정의
# => parent_splitter 추가 
retriever = ParentDocumentRetriever(
    # 벡터 저장소를 지정합니다.
    vectorstore=vectorstore,
    # 문서 저장소를 지정합니다.
    docstore=store,
    # 하위 문서 분할기를 지정합니다.
    child_splitter=child_splitter,
    # 상위 문서 분할기를 지정합니다.
    parent_splitter=parent_splitter,
)

retriever.add_documents(docs)  # 문서를 retriever에 추가합니다.


In [3]:
# 유사도 검색 수행
query = "아버지 돌아가시면 유급휴가는 며칠인가요?"
sub_docs = vectorstore.similarity_search(query)
for sub_doc in sub_docs:
    print(f'{sub_doc.page_content}', end="\n=================================\n")

5. 결혼퇴직
결혼퇴직의 경우 퇴직 1개월 이내에 결혼할 시에는 위의 기준에 의거하여 지급한다.
6. 기타
경조금 신청 시 휴가 신청도 같이 진행해야 함이 원칙임
특별한 경우 사업부장 합의 시 경조금 신청일 이후 신청 가능(6개월 이내)
분할 사용 불가, 발생일 이전 신청 불가.
부 칙
(시행일) 이 규정은 2007년 9월 14일부터 시행한다.
5.8.2 일신상 상병으로 인한 휴직의 경우에는 최초 3개월간 월급여의 100% 지급하며, 그 후 3개
월은 월급여의 3/2, 나머지 3개월은 월급여액의 1/3을 지급한다.
단, 제 3자와의 합의 등으로 급여에 준하는 보상을 받았거나 받기로 합의된 경우에는 그 금액
을 공제한 금액으로 한다.
않는다.
5.12 겸직자 급여
직원이 타직을 겸직한 경우에는 그 겸직에 따라 변동되는 수당을 지급할 수 있다.
5.13 전임·파견, 복귀자 급여
전임·파견, 복귀자 급여는 발령일을 기준으로 매월 20일까지는 신근무지에서, 21일 이후에는 전근
무지에서 지급한다.
5.14 휴업지불
8.3 상여급 지급기준(액)은 별도로 정한다.
8.4 상여금은 발령일을 기준으로 금액을 일할 계산하여 지급한다. 이 때, 휴직기간은 일할 계산에서 제외한다.
9. 퇴직금
9.1 지급대상
퇴직금은 만 1년이상 근속한 직원이 퇴직하였을 경우에 지급한다. 
9.2 근속기간
9.2.1 근속기간은 입사일로부터 퇴직전일까지로 한다


In [4]:
# 문서를 검색하여 가져옵니다.
retrieved_docs = retriever.get_relevant_documents(query)

print(f'*검색된 문서 계수:{len(retrieved_docs)}',end="\n\n=====================\n\n")

# 검색된 문서의 문서의 페이지 내용의 길이를 출력합니다.
for i, retrieved_doc in enumerate(retrieved_docs): 
    retrieved_doc_len = len(retrieved_doc.page_content)
    print(f"*문서{i+1}의 길이: {retrieved_doc_len}", end="\n\n\n")
    # 문서의 일부를 출력합니다.
    print(retrieved_doc.page_content[:100], end="\n\n=====================\n\n")


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


*검색된 문서 계수:4


*문서1의 길이: 369


1,000,000
1,000,000 500,000
300,000
200,000
조화 지급
조화 지급
조화 지급
조화 지급
조화 지급
조화 지급
3. 신청서류
경조금 : 경조금지급신


*문서2의 길이: 352


5.7.2 신규채용되어 수습기간중에 있는 자는 월급여의 90%를 지급한다.
5.7.3 복직자의 급여는 휴직당시 직급·호봉에 해당하는 현급여액을 지급함을 원칙으로 하나, 별도
의 정


*문서3의 길이: 370


않는다.
5.12 겸직자 급여
직원이 타직을 겸직한 경우에는 그 겸직에 따라 변동되는 수당을 지급할 수 있다.
5.13 전임·파견, 복귀자 급여
전임·파견, 복귀자 급여는 발령일을


*문서4의 길이: 384


8.3 상여급 지급기준(액)은 별도로 정한다.
8.4 상여금은 발령일을 기준으로 금액을 일할 계산하여 지급한다. 이 때, 휴직기간은 일할 계산에서 제외한다.
9. 퇴직금
9.1 지


